# Импорт библиотек

In [1]:
import pandas as pd

# Загрузка данных

In [2]:
# извне заданные данные (два списка размеров устриц в Петербурге и Магадане), 
# размеры списков разные
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689]

# объединение списков в DataFrame (создание двух DataFrame и их объединение)
petersburg_df = pd.DataFrame({'petersburg': petersburg})
magadan_df = pd.DataFrame({'magadan': magadan})
oysters = pd.concat([petersburg_df, magadan_df], axis=1)

# проверка результата
oysters

,petersburg,magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,NaN,0.0689


# Проверка данных на нормальность

In [3]:
# установка гипотез (нулевой и альтернативной)
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

In [4]:
# установка уровня значимости
alpha = 0.05

### Тест Шапиро-Уилка

In [5]:
# импорт теста Шапиро-Уилка
from scipy.stats import shapiro

In [6]:
# выполнение теста для первой группы 'petersburg', получаем интересующее p-value
# причем берется массив без последнего элемента, т.к. тот = NaN
stat, p = shapiro(oysters.loc[0:6, 'petersburg'])
print('Statistics=%.3f, p-value=%.3f' % (stat, p))

# Интерпретация 
if p > alpha:
	print(H0)
else:
	print(Ha)

Statistics=0.883, p-value=0.242
Данные распределены нормально


In [7]:
# выполнение теста для второй группы 'magadan', получаем интересующее p-value
stat, p = shapiro(oysters['magadan'])
print('Statistics=%.3f, p-value=%.3f' % (stat, p))

# Интерпретация 
if p > alpha:
	print(H0)
else:
	print(Ha)

Statistics=0.809, p-value=0.036
Данные не распределены нормально (мы отвергаем H0)


### Тест Д’Агостино

In [8]:
# импорт теста Д'Агостино
from scipy.stats import normaltest

In [9]:
# выполнение теста для первой группы 'petersburg', 
# к сожалению, в состав данного теста входит skewtest, который не может быть
# выполнен для массива с менее, чем 8 элементами (существующими)
# stat, p = normaltest(oysters['petersburg'], nan_policy='omit')

In [10]:
# выполнение теста для второй группы 'magadan', получаем интересующее p-value
stat, p = normaltest(oysters['magadan'])
print('Statistics=%.3f, p-value=%.3f' % (stat, p))

# Интерпретация 
if p > alpha:
	print(H0)
else:
	print(Ha)

Statistics=3.738, p-value=0.154
Данные распределены нормально


c:\Users\artem\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_stats_py.py:1772: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


### Тест Андерсона-Дарлинга

In [11]:
# импорт теста Андерсона-Дарлинга
from scipy.stats import anderson

In [12]:
# выполнение теста для первой группы 'petersburg', 
# получаем интересующее p-value, 
# причем берется массив без последнего элемента, т.к. тот = NaN
result = anderson(oysters.loc[0:6, 'petersburg'])
print('Statistic: %.3f' % result.statistic)

for i in range(len(result.critical_values)):
	sl, cv = result.significance_level[i], result.critical_values[i]
	if result.statistic < result.critical_values[i]:
		print(f'Уровень значимости: {int(sl)}%, p-value: {cv},', H0)
	else:
		print(f'Уровень значимости: {int(sl)}%, p-value: {cv},', Ha)

Statistic: 0.497
Уровень значимости: 15%, p-value: 0.543, Данные распределены нормально
Уровень значимости: 10%, p-value: 0.618, Данные распределены нормально
Уровень значимости: 5%, p-value: 0.742, Данные распределены нормально
Уровень значимости: 2%, p-value: 0.865, Данные распределены нормально
Уровень значимости: 1%, p-value: 1.029, Данные распределены нормально


In [13]:
# выполнение теста для второй группы 'magadan', получаем интересующее p-value
result = anderson(oysters['magadan'])
print('Statistic: %.3f' % result.statistic)

for i in range(len(result.critical_values)):
	sl, cv = result.significance_level[i], result.critical_values[i]
	if result.statistic < result.critical_values[i]:
		print(f'Уровень значимости: {int(sl)}%, p-value: {cv},', H0)
	else:
		print(f'Уровень значимости: {int(sl)}%, p-value: {cv},', Ha)

Statistic: 0.691
Уровень значимости: 15%, p-value: 0.519, Данные не распределены нормально (мы отвергаем H0)
Уровень значимости: 10%, p-value: 0.591, Данные не распределены нормально (мы отвергаем H0)
Уровень значимости: 5%, p-value: 0.709, Данные распределены нормально
Уровень значимости: 2%, p-value: 0.827, Данные распределены нормально
Уровень значимости: 1%, p-value: 0.984, Данные распределены нормально


### Вывод о нормальности данных

На основе трех проведенных тестов "на нормальность" не представляется сделать твердый, уверенный, однозначный вывод о нормальном распределении имеющихся данных. В частности это связано с малым количеством образцов(7 и 8).
Приблизительно, можно принять, что распределение данных близко к нормальному.

## Выявление наличия корреляции

Для выявления возможной корреляции следует использовать непараметрический метод (в связи с отсутствием твердой уверенности в нормальном распределении данных).

###  Тест на корреляцию Спирмена

In [14]:
# импорт теста Спирмена
from scipy.stats import spearmanr

In [15]:
# выполнение теста , причем игнорируется значение NaN
rho, p = spearmanr(oysters, nan_policy='omit')

# вывод результата, причем
# p-value неинформативен, т.к. точность зависит от 
# размера выборки (требуется от 500)
print('Коэффициент корреляции = %.3f' % rho)
print('p-value = %.3f' % p)

Коэффициент корреляции = 0.143
p-value = 0.760


### Тест на корреляцию Кендалла

In [16]:
# импорт теста Кендалла
from scipy.stats import kendalltau

In [17]:
# выполнение теста , причем игнорируется значение NaN
coef, p = kendalltau(oysters['petersburg'], oysters['magadan'], nan_policy='omit')

# вывод результата
print('Коэффициент корреляции = %.3f' % coef)
print('p-value = %.3f' % p)

Коэффициент корреляции = 0.143
p-value = 0.773


### Дополнительно

Т.к. ранее сказано: "Приблизительно, можно принять, что распределение данных близко к нормальному" - проверим наличие корреляции с помощью параметрического теста на корреляцию Пирсона.

### Тест на корреляцию Пирсона

In [18]:
# импорт теста Пирсона
from scipy.stats import pearsonr

In [19]:
# данный тест не работает с выборками с наличием NaN,
# заменим последний элемент для petersburg на медиану,
# с учетом того, что принимаем эту выборку за нормально распределенную
oysters_copy = oysters.copy()
oysters_copy.loc[7, 'petersburg'] = oysters_copy['petersburg'].median()

# выполнение теста 
corr, p = pearsonr(oysters_copy['petersburg'], oysters_copy['magadan'])

# вывод результата
print('Коэффициент корреляции = %.3f' % corr)
print('p-value = %.3f' % p)

Коэффициент корреляции = 0.257
p-value = 0.539


### Вывод о наличии корреляции

Оба непараметрических теста показали коэффициент корреляции 0.143. Что соответствует прямой очень слабой связи. Параметрический тест также показал схожий результат 0.257.
Можно считать, что корреляция между двумя проверяемыми массивами отсутствует (настолько мала, что ее можно не принимать в расчет).
Дополнительно можно обратить внимание на величины p-value, они велики: 0.54, 0.76, 0.77. Это также свидетельствует против наличия корреляции между имеющимися выборками.

## Какова разница в среднем размере мидии в зависимости от города-производителя

 Для данного исследования будет использован непараметрический тест - ранговый тест Манна-Уитни (тест суммы рангов Уилкоксона). Выбор сязан с тем, что имеется именно пара выборок, выборки независимы, размеры выборок малы (n < 30) и распределение считается отличным от нормального.

### Tест Манна-Уитни

In [20]:
# импорт теста Манна-Уитни
from scipy.stats import mannwhitneyu

In [21]:
# выполнение теста, причем игнорируется значение NaN
Up, p = mannwhitneyu(oysters.loc[0:6, 'petersburg'], oysters['magadan'])

# вывод результата
# статистика вычислена для распределения 'petersburg' 
# (для первой выборки)
print('Up-статистика Манна-Уитни для распределения petersburg = %.0f' % Up)
print('p-value = %.3f' % p)

Up-статистика Манна-Уитни для распределения petersburg= 51
p-value = 0.006


In [22]:
# вычисление статистики для распределения 'magadan'
nx, ny = len(oysters.loc[0:6, 'petersburg']), len(oysters['magadan'])
Um = nx*ny - Up
print('Um-статистика Манна-Уитни для распределения magadan = %.0f' % Um)

Um-статистика Манна-Уитни для распределения magadan = 5


In [23]:
# U-статистика Манна-Уитни
U = min(Um, Up)
print('U-статистика Манна-Уитни = %.0f' % U)

U-статистика Манна-Уитни = 5


### Интерпретация теста

Нулевая гипотеза для рангового теста Манна-Уитни заключается в том, что распределение, лежащее в основе выборки первой выборки, совпадает с распределением, лежащим в основе второй выборки.

В данном случае нулевая гипотеза заключается в совпадении средних значений двух независимых выборок (средних размеров мидий в двух городах-производителях). Альтернативная гипотеза - средние значения двух выборок различны (существенно отличаются). Уровень статистической значимости примем 0.05.

По результатам теста получено значение U-статистики (критерия) Манна-Уитни: 5. По таблице критических значений, для выбранного уровня статистической значимости 0.05, определено критическое значение U-критерия Манна-Уитни для количества элементов первой и второй выборки: 10. Т.о. полученное значение U-статистики меньше табличного (5 < 10) (существенно меньше).  Следовательно признается наличие существенного различия между уровнем признака (среднее значение) в рассматриваемых выборках (принимается альтернативная гипотеза). 

Следует отметить значение p-value равное 0.006. Соответственно полученные результаты можно считать статистически значимыми. И данное значение также подтверждает отклонение нулевой гипотезы.

### Дополнительно

Т.к. ранее сказано: "Приблизительно, можно принять, что распределение данных близко к нормальному" - дополнительно используем t-критерий для двух независимых выборок.

### T-критерий для двух независимых выборок

In [24]:
# импорт теста t-критерий для двух независимых выборок
from scipy.stats import ttest_ind

In [25]:
# выполнение теста, причем игнорируется значение NaN,
# кроме того выполнение именно t-критерия Уэлча,
# (equal_var=False) т.е. считаем, что дисперсии 
# элементов выборок не равны
stat, p = ttest_ind(oysters['petersburg'], oysters['magadan'], nan_policy='omit', equal_var=False)
print('t-статистика = %.3f' % stat)
print('p-value = %.3f' % p)

t-статистика = 3.325
p-value = 0.006


### Интерпретация теста

Данный тест проверяет нулевую гипотезу о том, что две независимые выборки имеют одинаковые средние (ожидаемые) значения. 

Т.е. нулевая гипотеза: средние значения двух независимых выборок (средние размеры мидий в двух городах-производителях) совпадают. Альтернативная гипотеза - средние значения двух выборок различны (существенно отличаются). Уровень статистической значимости примем 0.05.

По результатам теста получаем значение p-value = 0.006. Т.о. результаты являются статистически значимыми и нулевая гипотеза отклоняется. Соответственно, принимается альтернативная гипотеза - средние значения существенно отличаются.

Дополнительно. Если принять, что дисперсии элементов выборок равны (t-критерий Стьюдента), то p-value = 0.005. Т.е. результат неизменен.

## Разница в среднем размере мидии в зависимости от города-производителя - **существенно различна**.